In [1]:
import numpy as np 
import pandas as pd 

In [50]:
spotify = pd.read_csv('Spotify_Attribute_Data.csv')
spotify.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [51]:
#Modify artists column so only main artist is in list and not features
spotify['artists'] = spotify['artists'].apply(lambda x: x.replace('[','').replace(']','').replace('""',''))
spotify['artist'] = spotify['artists'].str.split(',', expand = True)[0]

In [52]:
#Drop unrequired columns and sort columns
spotify.drop(columns=['artists', 'release_date', 'id'])
spotify = spotify[['name', 'artist', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
                  'popularity', 'speechiness', 'tempo', 'valence']]

In [53]:
#Convert time column from milliseconds to seconds
spotify['duration_ms'] = spotify['duration_ms'] / 1000

In [54]:
#Rename time column
spotify.rename(columns={"duration_ms": "duration_s"})

,name,artist,year,acousticness,danceability,duration_s,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Keep A Song In Your Soul,'Mamie Smith',1920,0.991000,0.598,168.333,0.224,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340
1,I Put A Spell On You,"""Screamin' Jay Hawkins""",1920,0.643000,0.852,150.200,0.517,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500
2,Golfing Papa,'Mamie Smith',1920,0.993000,0.647,163.827,0.186,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890
3,True House Music - Xavier Santos & Carlos Gomi...,'Oscar Velazquez',1920,0.000173,0.730,422.087,0.798,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422
4,Xuniverxe,'Mixe',1920,0.295000,0.704,165.224,0.707,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,The One,'DJ Combo',2020,0.009170,0.792,147.615,0.866,0,0.000060,6,0.1780,-5.089,0,0,0.0356,125.972,0.1860
174385,A Little More,'Alessia Cara',2021,0.795000,0.429,144.720,0.211,0,0.000000,4,0.1960,-11.665,1,0,0.0360,94.710,0.2280
174386,Together,'Roger Fly',2020,0.806000,0.671,218.147,0.589,0,0.920000,4,0.1130,-12.393,0,0,0.0282,108.058,0.7140
174387,champagne problems,'Taylor Swift',2021,0.920000,0.462,244.000,0.240,1,0.000000,0,0.1130,-12.077,1,69,0.0377,171.319,0.3200




Data has duplicate entries of songs. Code below is to remove duplicates and leave only row with max popoularity

In [55]:
spotify['artists+name'] = spotify.apply(lambda row: row['artist'] + row['name'], axis=1)

In [56]:
duplicated_df = spotify[spotify['artists+name'].duplicated()]
duplicated_df.shape

#16107 duplicates

(16107, 18)

In [57]:
# We gather the list of indices corresponding to the maximum popularity for each duplicated artist/song pair.
indices = []

for name in duplicated_df['artists+name'].unique():
    subset = spotify[spotify['artists+name'] == name].copy()
    m = subset['popularity'].max()
    index = subset[subset['popularity'] == m].index[0]
    indices.append(index)

In [58]:
data_bis = spotify.loc[indices].copy()
data_bis['artists+name'].duplicated().sum()

0

In [61]:
data_ter = spotify.copy()
for i, row in spotify.iterrows():
    if row['artists+name'] in duplicated_df['artists+name'].unique():
        data_ter.drop(index=i, inplace=True)

In [62]:
frames = [data_bis, data_ter]
clean_spotify = pd.concat(frames)

In [63]:
clean_spotify['artists+name'].duplicated().sum()

0

In [64]:
clean_spotify.drop(columns=['artists+name'], inplace=True)

In [65]:
#Remove null rows
clean_spotify = clean_spotify[clean_spotify['tempo'] != 0].copy()

In [66]:
clean_spotify

,name,artist,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
64,Argilamas kai psarades,'Amalia Vaka',1920,0.995000,0.287,216.720,0.6210,0,0.7840,3,0.1510,-10.843,0,0,0.0394,77.537,0.5590
82,Sti filaki me valane,'Marika Papagkika',1920,0.932000,0.382,231.400,0.4920,0,0.0137,1,0.1570,-8.321,1,0,0.0345,88.525,0.6400
111,To ouest,'Xaralampos Kritikos',1920,0.895000,0.474,227.320,0.4440,0,0.0000,6,0.1660,-9.783,1,0,0.0378,80.317,0.9200
14,Mparmpaouzos,'Takis Nikolaou',1920,0.993000,0.453,255.520,0.4100,0,0.1600,2,0.2920,-10.469,1,0,0.0440,67.271,0.8940
16,Oli mera paizei zaria,'Giorgos Katsaros',1920,0.972000,0.528,277.720,0.3250,0,0.0234,11,0.1020,-12.508,1,0,0.0421,118.562,0.1460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174357,So Fine - TC Remix,'Sean Paul',2021,0.000032,0.530,292.427,0.9590,0,0.0273,7,0.3550,-5.648,1,0,0.1440,173.009,0.2620
174361,Little Boy,'Ashnikko',2021,0.105000,0.781,172.720,0.4870,1,0.0000,1,0.0802,-7.301,0,61,0.1670,129.941,0.3270
174365,I giorni,'Ludovico Einaudi',2021,0.976000,0.358,381.150,0.1210,0,0.8890,2,0.1350,-25.111,1,0,0.0532,131.808,0.1060
174369,Nuvole bianche,'Ludovico Einaudi',2021,0.995000,0.297,349.200,0.0287,0,0.9080,8,0.0995,-30.008,1,0,0.0564,141.636,0.0678


In [70]:
clean_spotify.to_csv(r'C:\Users\sasit\OneDrive\Desktop\Data_Coursework\Project\final-project\Clean_Spotify_Attribute_Data.csv', index = None, header=True)

In [72]:
clean_spotify2020 = clean_spotify.loc[clean_spotify['year'] == 2020]

In [73]:
clean_spotify2020.to_csv(r'C:\Users\sasit\OneDrive\Desktop\Data_Coursework\Project\final-project\2020only_Clean_Spotify_Attribute_Data.csv', index = None, header=True)

In [2]:
spotifyyear = pd.read_csv(r'C:\Users\sasit\OneDrive\Desktop\Data_Coursework\Project\final-project\Clean_Spotify_Attribute_Data.csv')

In [6]:
spotifyyear = spotifyyear.drop(columns=['name', 'artist', 'key'])
spotifyyear

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,1920,0.995000,0.287,216.720,0.6210,0,0.7840,0.1510,-10.843,0,0,0.0394,77.537,0.5590
1,1920,0.932000,0.382,231.400,0.4920,0,0.0137,0.1570,-8.321,1,0,0.0345,88.525,0.6400
2,1920,0.895000,0.474,227.320,0.4440,0,0.0000,0.1660,-9.783,1,0,0.0378,80.317,0.9200
3,1920,0.993000,0.453,255.520,0.4100,0,0.1600,0.2920,-10.469,1,0,0.0440,67.271,0.8940
4,1920,0.972000,0.528,277.720,0.3250,0,0.0234,0.1020,-12.508,1,0,0.0421,118.562,0.1460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158165,2021,0.000032,0.530,292.427,0.9590,0,0.0273,0.3550,-5.648,1,0,0.1440,173.009,0.2620
158166,2021,0.105000,0.781,172.720,0.4870,1,0.0000,0.0802,-7.301,0,61,0.1670,129.941,0.3270
158167,2021,0.976000,0.358,381.150,0.1210,0,0.8890,0.1350,-25.111,1,0,0.0532,131.808,0.1060
158168,2021,0.995000,0.297,349.200,0.0287,0,0.9080,0.0995,-30.008,1,0,0.0564,141.636,0.0678


In [7]:
spotifyyeargroup = spotifyyear.groupby(['year']).mean()

In [19]:
spotifyyeargroup = spotifyyeargroup.reset_index()

In [24]:
spotifyyeargroup = spotifyyeargroup[spotifyyeargroup['year'] != 2021]

In [25]:
spotifyyeargroup

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,1920,0.609566,0.528261,240.697910,0.418317,0.133956,0.352186,0.216945,-12.780682,0.623053,0.663551,0.086487,115.058813,0.494858
1,1921,0.854134,0.446471,246.787552,0.252671,0.075862,0.298936,0.207768,-16.238483,0.668966,0.413793,0.081846,103.739807,0.400629
2,1922,0.826143,0.579193,139.860664,0.228543,0.000000,0.245192,0.259093,-20.765882,0.663866,0.092437,0.470957,100.308756,0.573571
3,1923,0.956123,0.573095,176.925179,0.265430,0.000000,0.366214,0.236029,-14.228952,0.785714,5.214286,0.094787,112.498857,0.621733
4,1924,0.937941,0.557674,192.950073,0.353079,0.000000,0.585091,0.235738,-14.102394,0.747706,0.711009,0.095224,121.969720,0.674428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2016,0.232287,0.585353,255.312044,0.653940,0.191388,0.193623,0.221964,-7.955402,0.633971,31.773206,0.099637,124.025456,0.435439
97,2017,0.201982,0.582812,241.292921,0.689394,0.235531,0.237164,0.231029,-7.916047,0.609965,33.206341,0.098815,122.353357,0.436774
98,2018,0.230667,0.602959,224.369876,0.660208,0.255654,0.226433,0.234826,-8.160672,0.581179,28.844911,0.125115,123.786570,0.420005
99,2019,0.266461,0.606619,222.655654,0.628814,0.254589,0.210679,0.219892,-8.464186,0.600966,34.639130,0.110248,122.486747,0.457644


In [27]:
spotifyyeargroup.to_csv(r'C:\Users\sasit\OneDrive\Desktop\Data_Coursework\Project\final-project\groupedbyyear_attributes.csv', index = None, header=True)

In [3]:
spotifyyear

,name,artist,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Argilamas kai psarades,'Amalia Vaka',1920,0.995000,0.287,216.720,0.6210,0,0.7840,3,0.1510,-10.843,0,0,0.0394,77.537,0.5590
1,Sti filaki me valane,'Marika Papagkika',1920,0.932000,0.382,231.400,0.4920,0,0.0137,1,0.1570,-8.321,1,0,0.0345,88.525,0.6400
2,To ouest,'Xaralampos Kritikos',1920,0.895000,0.474,227.320,0.4440,0,0.0000,6,0.1660,-9.783,1,0,0.0378,80.317,0.9200
3,Mparmpaouzos,'Takis Nikolaou',1920,0.993000,0.453,255.520,0.4100,0,0.1600,2,0.2920,-10.469,1,0,0.0440,67.271,0.8940
4,Oli mera paizei zaria,'Giorgos Katsaros',1920,0.972000,0.528,277.720,0.3250,0,0.0234,11,0.1020,-12.508,1,0,0.0421,118.562,0.1460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158165,So Fine - TC Remix,'Sean Paul',2021,0.000032,0.530,292.427,0.9590,0,0.0273,7,0.3550,-5.648,1,0,0.1440,173.009,0.2620
158166,Little Boy,'Ashnikko',2021,0.105000,0.781,172.720,0.4870,1,0.0000,1,0.0802,-7.301,0,61,0.1670,129.941,0.3270
158167,I giorni,'Ludovico Einaudi',2021,0.976000,0.358,381.150,0.1210,0,0.8890,2,0.1350,-25.111,1,0,0.0532,131.808,0.1060
158168,Nuvole bianche,'Ludovico Einaudi',2021,0.995000,0.297,349.200,0.0287,0,0.9080,8,0.0995,-30.008,1,0,0.0564,141.636,0.0678
